<html>
    <h1>Libraries used :</h1>
    <ul style="list-style-type: none;">
        <li>
             <img width=150px height=150px  src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Sympy_logo.svg/1200px-Sympy_logo.svg.png"  alt="Sympy">
        </li>
        <br>
        <li>
             <img width=300px height=300px  src="https://matplotlib.org/_static/logo2.png" alt="Matplotlib">
        </li>
        <br>
        <li>
             <img width=200px height=200px src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/1200px-NumPy_logo.svg.png" alt="Numpy">
        </li>
        <br>
        <li>        
            <img width=200px height=200px  src="https://www.fullstackpython.com/img/logos/scipy.png" class="rounded" alt="Scipy">
        </li>
        <br>
        <li>
             <img width=400px height=400px  src="https://pandas.pydata.org/_static/pandas_logo.png" class="rounded" alt="Pandas">
        </li>
      </ul>

In [1]:
import numpy as np
from numba import jit, int32, float32, prange
from scipy.sparse import diags
from scipy.integrate import RK45, solve_ivp
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib
from time import time
import pandas as pd

import time
import os
import math
#import pandas as pd
from tqdm import tqdm_notebook


In [2]:
plt.ioff()
plt.clf()

<html>
    <h3 align="center" style="color:#FF1C0C;"><u> Heat Equation </u> </h3> 
   <br>
</html>

\begin{equation}
\\\ u_t  + u u_x + \beta u_{xx}    = - \left(\pi^{2} \beta - \pi e^{t} \cos{\left(\pi x \right)} - 1\right) e^{t} \sin{\left(\pi x \right)}
\\ u^h = \sum_{i= 0}^{n} a_i(t)  \phi_i(x) 
\\ \int_0^1 { (w u_t +u u_x+ \beta w u_{xx})} \ dx= \int_0^1 w (- \left(\pi^{2} \beta - \pi e^{t} \cos{\left(\pi x \right)} - 1\right) e^{t} \sin{\left(\pi x \right)}) dx
\\\ \int_0^1  (w u_t + uu_x)   \ dx   - \beta \int_0^1 {w_x u_x} \ dx  + \beta \ (w(1) u_x(1) - w(0) u_x(0))= \int_0^1 w (- \left(\pi^{2} \beta - \pi e^{t} \cos{\left(\pi x \right)} - 1\right) e^{t} \sin{\left(\pi x \right)}) dx
\\  A \alpha^{'}(t) \hspace{5mm} +\alpha^T B \alpha \ -  \hspace{5mm}\beta  D \alpha(t) \hspace{10mm} +\hspace{10mm}  \beta E \alpha(t)   = f(t)
\\ w = {\phi_j}(x)
\end{equation}

\begin{equation}
\\  \alpha^{'}(t) = A^{-1}( (-\alpha^T B  +\beta ( D - E)) \alpha(t) + f(t)) \hspace{10mm} 
\end{equation}


<html>
    <h3 align="center" style="color:#FF1C0C;"><u>Boundary Conditions</u> </h3> <br>
</html>
\begin{equation}
\\  u(0, t) = 0  
\\  u(N, t) = 0
\\  u(x, 0) = sin(\pi x) 
\end{equation}

<html>
    <h3 align="center" style="color:#FF1C0C;"><u>Exact Solution</u> </h3> <br>
</html>
\begin{equation}
\\  u(x, t) = e^t sin (\pi x) 
\end{equation}

In [3]:

class Heat:
    
    #A *alpha - beta(D *) +beta(E(1 - 0)alpha) = 0
    

    def __init__(self, N=10):
        
        self.N = N
        self.H = 1/N
        self.domain = np.linspace(0, 1, N+1) 
        self.beta = -1
        self.t = 1
        #Set initial conditions here
        self.alpha = np.sin(self.domain)
        #Set solution here
        self.exact = math.exp(self.t) * self.alpha
        #Define f
        def f(k, exp_t):
            if exp_t == 1:
                if k == 0:
                    return (-(self.H - math.sin(self.H))*(self.beta - 1)/self.H)
                elif k == N:
                    return ((self.beta - 1)*(self.H*math.cos(1) - math.sin(self.H - 1) - math.sin(1))/self.H)
                else:
                    return (2*(self.beta - 1)*(math.cos(self.H) - 1)*math.sin(self.H*k)/self.H)
            else:
                if k == 0:
                    return ((1/8)*(2*self.H - math.sin(2*self.H))/self.H)
                elif k == N:
                    return (-1/4*(self.H*math.cos(2) - math.sin(self.H)*math.cos(self.H - 2))/self.H)
                else:
                    return (math.sin(self.H)**2*math.sin(self.H*k)*math.cos(self.H*k)/self.H)
                
        #Define boundary conditions
        self.b_c_start = 0
        self.b_c_end = math.exp(1) * math.sin(1)
        
        def c_dash_transform(l):
            
            l[0] /= self.A[0]
            for i in range(1, self.N):
                  l[i] = (l[i])/ (self.A[i]-(self.H/6)* l[i-1])
            return l        
       
        def matB(i):
                if i == 0 :
                    row = np.array([0, 0, 1, 1])
                    col = np.array([0, 1, 0, 1])
                    data = np.array([-1/3, 1/3, -1/6, 1/6])
                    return sp.csr_matrix((data, (row, col)), shape=(N+1, N+1), dtype=np.float32)        
                elif i == self.N:
                    row = np.array([self.N-1, self.N - 1, self.N, self.N])
                    col = np.array([self.N-1, self.N , self.N -1, self.N])
                    data = np.array([-1/6, 1/6, -1/3, 1/3])
                    return sp.csr_matrix((data, (row, col)), shape=(N+1, N+1), dtype=np.float32)   
                else:
                    row = np.array([i-1]*3 + [i]*3 + [i+1]*3)
                    col = np.array([col for col in range(i-1, i+1 + 1)]*3, dtype=np.float32)
                    data = np.array([-1/6, 1/6, 0, -1/3, 0, 1/3, 0, -1/6, 1/6 ])
                    return sp.csr_matrix((data, (row, col)), shape=(N+1, N+1), dtype=np.float32)    

              
                
        self.A = self.H* np.array([1/3] + [2/3]*(N-1) + [1/3])
        #         self.A = (self.H) * sp.csc_matrix(diags([       [1/6 for i in range(N)],   \
        #                                         [1/3] + [2/3]*(N-1) + [1/3], \
        #                                              [1/6 for i in range(N)] ], \
        #                                                    [1, 0, -1]), dtype=np.float32)
        
        self.B = np.array([matB(i) for i in prange(N+1)])
        
        self.c_dash = c_dash_transform(self.H *np.array( [1/6]*(N)))
        
        #-beta*
        
        
        self.D = N * sp.csr_matrix(diags([      [-1 for i in range(N)],  \
                                      [1] + [2]*(N-1) + [1], \
                                        [-1 for i in range(N)]],  \
                             [1, 0, -1] ), dtype=np.float32)
        self.D[0, 0] -= N     
        self.D[0, 1] -= -N
        self.D[N, N-1] -= -N
        self.D[N, N] -= N
        
           
        self.F1 = np.array([f(k, 1) for k in prange(N+1)]).reshape(-1, 1) #incomplete without exp(t)
        self.F2 = np.array([f(k, 2) for k in prange(N+1)]).reshape(-1, 1) #incomplete without exp(t)


       
      
    def error(self):
        return np.linalg.norm(self.exact -  self.approx, 2)

    
    def apply_bc(self,  y):
        y[0, 0] = self.b_c_start
        y[-1, 0] = self.b_c_end
        return y
        
    
    def func (self, t,  y):
        d = (  sp.csr_matrix(np.vstack([-y.T.ravel() * i for i in self.B]))+ self.beta*self.D).dot(y) + math.exp(t)*(self.F1 + math.exp(t)*self.F2)
        
        #A *alpha' = d form 
        
        def simplifier( d):
            d[0] /= self.A[0]
            for i in range(1, self.N + 1):
                d[i, 0] = (d[i, 0] - (self.H/6)*d[i-1])/(self.A[i]-(self.H/6)*self.c_dash[i-1])
            for i in range(self.N-1, -1, -1): # Last entry is left unchanged ;see that i+1
                d[i, 0] -= self.c_dash[i]*d[i+1]                
        
            return self.apply_bc(d)                   
        
        #y = self.A*(self.beta*(self.D* y) + math.exp(t)*self.F)
        return simplifier(d)


    
    def algo_1(self):

        self.algo = solve_ivp(self.func,t_span=(0, 1),  y0=self.alpha, \
                   t_eval= np.array([1]), vectorized=True, method="LSODA")
        self.approx = self.apply_bc(self.algo.y).reshape(-1, )
        

In [4]:
Ns = [2**r for r in range(2, 9)]
l =[]
for i in tqdm_notebook(Ns):
    heateqn = Heat(i)

    t0 = time.time()

    heateqn.algo_1()
    l.append([i, heateqn.error()])
    if i!= Ns[-1]:
        del heateqn
    print(f"Time taken for algo to run for {i} basis",  time.time()-t0)
    


Time taken for algo to run for 4 basis 0.17670798301696777
Time taken for algo to run for 8 basis 0.41062188148498535
Time taken for algo to run for 16 basis 0.9757339954376221
Time taken for algo to run for 32 basis 2.871732234954834
Time taken for algo to run for 64 basis 10.109773874282837
Time taken for algo to run for 128 basis 39.65156531333923
Time taken for algo to run for 256 basis 156.05403351783752



In [8]:
%matplotlib widget
errrs = []
xs = []
for i, j in l:
    xs.append(i)
    errrs.append(j)

xs, errrs =  np.array(xs), np.array(errrs)
slope, intercept = np.polyfit(-np.log(xs), np.log(errrs), 1)

plt.loglog(1/np.array(xs), errrs, '--')
plt.xlabel("H")
plt.ylabel("Error")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Error')

In [9]:
print(slope)

-0.5699007503919558


In [10]:
df = pd.DataFrame(data = {"N":xs , "H": 1/xs, "Errors":errrs})
df.to_csv("Non Lin Non Hom.csv")